In [1]:
import torch
import emoji
import os
import sys
import pandas as pd
import numpy as np
import scipy.stats as ss
sys.path.insert(0, '../')

from tqdm import tqdm
from config import GPT2EmojiConfig
from model import GPT2LMEmojiModel
from transformers import GPT2Tokenizer
from run_language_modeling import load_and_cache_examples, targets_mask
from sst_binary import sst_binary

MODEL_CLASSES = {
    "gpt2": (GPT2EmojiConfig, GPT2LMEmojiModel, GPT2Tokenizer),
}

MODEL_PATH = '../checkpoint-180000'

args = torch.load(os.path.join(MODEL_PATH, 'training_args.bin'))

config_class, model_class, tokenizer_class = MODEL_CLASSES['gpt2']

config = config_class.from_pretrained(MODEL_PATH)

tokenizer = tokenizer_class.from_pretrained(MODEL_PATH)

model = model_class.from_pretrained(
            MODEL_PATH,
            config=config,
)

map_target_to_token_id = dict(
        zip(range(0, len(emoji.UNICODE_EMOJI.keys())), tokenizer.encode(list(emoji.UNICODE_EMOJI.keys())))
)

Token indices sequence length is longer than the specified maximum sequence length for this model (2811 > 1024). Running this sequence through the model will result in indexing errors


In [2]:
trX, vaX, teX, trY, vaY, teY = sst_binary()
# _, vaX, _, _, vaY, _ = sst_binary()

In [3]:
def encode(X, tokenizer, block_size=512):
    return tokenizer.batch_encode_plus(X, add_special_tokens=True, max_length=block_size)["input_ids"]

def predict_mean(X_ids, model):
    preds = []
    for x in tqdm(X_ids):
        outputs = model(torch.tensor(x).unsqueeze(0))
        logits = outputs[0].squeeze(0)
        
        target_id = logits.mean(dim=0).argmax().item()
        token_id = map_target_to_token_id[target_id]
        token = tokenizer.decode(token_id)
        
        preds.append(token)
    return preds

def predict_last(X_ids, model):
    preds = []
    for x in tqdm(X_ids):
        outputs = model(torch.tensor(x).unsqueeze(0))
        logits = outputs[0].squeeze(0)
        
        target_id = logits[-1].argmax().item()
        token_id = map_target_to_token_id[target_id]
        token = tokenizer.decode(token_id)
        
        preds.append(token)
    return preds

def predict_max(X_ids, model):
    preds = []
    for x in tqdm(X_ids):
        outputs = model(torch.tensor(x).unsqueeze(0))
        logits = outputs[0].squeeze(0)
        
        target_id = logits.max(0)[0].argmax().item()
        token_id = map_target_to_token_id[target_id]
        token = tokenizer.decode(token_id)
        
        preds.append(token)
    return preds

def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

In [4]:
vaX_ids = encode(vaX, tokenizer, block_size=args.block_size)

In [5]:
vaX_preds = predict_mean(vaX_ids, model)

100%|██████████| 872/872 [01:34<00:00,  9.25it/s]


In [6]:
confusion_matrix = pd.crosstab(pd.Series(vaX_preds), pd.Series(vaY))
confusion_matrix

col_0,0,1
row_0,,
✨,0,11
❤,0,4
🎶,0,1
🐾,0,1
👀,2,2
👇,3,5
👌,0,2
👍,7,25
👏,3,37


In [7]:
cramers_corrected_stat(confusion_matrix.to_numpy())

0.3603607194991182

In [8]:
vaX_preds = predict_last(vaX_ids, model)

confusion_matrix = pd.crosstab(pd.Series(vaX_preds), pd.Series(vaY))
confusion_matrix

100%|██████████| 872/872 [01:26<00:00, 10.05it/s]


col_0,0,1
row_0,,
✨,4,11
❤,0,13
➡,0,1
🌊,2,1
🌞,1,0
🌱,0,1
🍷,1,0
🍿,1,2
🎶,0,1


In [9]:
cramers_corrected_stat(confusion_matrix.to_numpy())

0.4830754094798683

In [10]:
vaX_preds = predict_max(vaX_ids, model)

confusion_matrix = pd.crosstab(pd.Series(vaX_preds), pd.Series(vaY))
confusion_matrix

100%|██████████| 872/872 [01:28<00:00,  9.86it/s]


col_0,0,1
row_0,,
✅,1,0
✨,0,8
❤,0,2
➡,0,1
🌊,4,1
🌍,1,3
🌟,0,1
🍆,2,2
🍕,1,0


In [11]:
cramers_corrected_stat(confusion_matrix.to_numpy())

0.29269319257713244

In [12]:
def predict_topk(X_ids, model, k=5):
    preds = []
    for x in tqdm(X_ids):
        outputs = model(torch.tensor(x).unsqueeze(0))
        logits = outputs[0].squeeze(0)
        
#         target_ids = logits[-1].topk(k)[1].tolist()
        target_ids = logits.mean(0).topk(k)[1].tolist()
        token_ids = [map_target_to_token_id[target_id] for target_id in target_ids]
        tokens = tokenizer.decode(token_ids)
        
        preds.append(tokens.split())
    return preds

def predict_topk_last(X_ids, model, k=5):
    preds = []
    for x in tqdm(X_ids):
        outputs = model(torch.tensor(x).unsqueeze(0))
        logits = outputs[0].squeeze(0)
        
        target_ids = logits[-1].topk(k)[1].tolist()
        token_ids = [map_target_to_token_id[target_id] for target_id in target_ids]
        tokens = tokenizer.decode(token_ids)
        
        preds.append(tokens.split())
    return preds

def predict_topk_last_second(X_ids, model):
    preds = []
    for x in tqdm(X_ids):
        outputs = model(torch.tensor(x).unsqueeze(0))
        logits = outputs[0].squeeze(0)
        
        target_id = logits[-1].topk(2)[1].tolist()[1]
        token_id = map_target_to_token_id[target_id]
        token = tokenizer.decode(token_id)
        
        preds.append(token)
    return preds

In [13]:
vaX_preds = predict_topk(vaX_ids, model, k=2)

df = pd.DataFrame(vaX_preds)

confusion_matrix = pd.crosstab([df[col] for col in df.columns], pd.Series(vaY))

cramers_corrected_stat(confusion_matrix.to_numpy())

100%|██████████| 872/872 [01:50<00:00,  7.88it/s]


0.4876394261178236

In [14]:
vaX_preds = predict_topk(vaX_ids, model, k=3)

100%|██████████| 872/872 [01:26<00:00, 10.04it/s]


In [15]:
df = pd.DataFrame(vaX_preds)
df.head()

,0,1,2
0,😂,😭,😍
1,😂,🙄,😉
2,👇,😂,👍
3,😂,👍,🔥
4,😂,👇,✨


In [16]:
confusion_matrix = pd.crosstab([df[col] for col in df.columns], pd.Series(vaY))
confusion_matrix

col_0  0  1
0 1 2      
✨ 👇 🔥  0  1
  👍 👌  0  1
  👏 ❤  0  1
    🔥  0  1
  😂 😁  0  1
...   .. ..
🙏 ✨ 💯  0  1
  👏 ✨  0  1
  😂 👇  1  0
    👍  0  1
    💯  0  1

[288 rows x 2 columns]

In [17]:
cramers_corrected_stat(confusion_matrix.to_numpy())

0.43417870979029327

In [18]:
vaX_preds = predict_topk_last(vaX_ids, model, k=2)

df = pd.DataFrame(vaX_preds)

confusion_matrix = pd.crosstab([df[col] for col in df.columns], pd.Series(vaY))

cramers_corrected_stat(confusion_matrix.to_numpy())

100%|██████████| 872/872 [01:28<00:00,  9.86it/s]


0.47409533429449197

In [19]:
vaX_preds = predict_topk_last_second(vaX_ids, model)

df = pd.DataFrame(vaX_preds)

confusion_matrix = pd.crosstab([df[col] for col in df.columns], pd.Series(vaY))
confusion_matrix

100%|██████████| 872/872 [01:29<00:00,  9.75it/s]


col_0,0,1
0,,
✨,1,5
❤,0,12
➡,0,2
🌕,0,1
🌟,0,2
🎶,1,0
🏀,1,0
🏆,1,0
👀,10,8


In [20]:
cramers_corrected_stat(confusion_matrix.to_numpy())

0.42458399802001184

In [21]:
trX_ids = encode(trX, tokenizer, block_size=args.block_size)

In [22]:
trX_preds = predict_topk(trX_ids, model, k=2)

df = pd.DataFrame(trX_preds)

confusion_matrix = pd.crosstab([df[col] for col in df.columns], pd.Series(trY))

cramers_corrected_stat(confusion_matrix.to_numpy())

  6%|▌         | 429/6920 [00:37<14:30,  7.46it/s]

KeyboardInterrupt: 

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(confusion_matrix)

In [ ]:
trX_preds = predict_last(trX_ids, model)

confusion_matrix = pd.crosstab(pd.Series(trX_preds), pd.Series(trY))

cramers_corrected_stat(confusion_matrix.to_numpy())

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(confusion_matrix)

In [56]:
#Entropy
def entropy(Y):
    """
    Also known as Shanon Entropy
    Reference: https://en.wikipedia.org/wiki/Entropy_(information_theory)
    """
    unique, count = np.unique(Y, return_counts=True, axis=0)
    prob = count/len(Y)
    en = np.sum((-1)*prob*np.log2(prob))
    return en

#Joint Entropy
def joint_entropy(Y,X):
    """
    H(Y;X)
    Reference: https://en.wikipedia.org/wiki/Joint_entropy
    """
    YX = np.c_[Y,X]
    return entropy(YX)

#Conditional Entropy
def conditional_entropy(Y, X):
    """
    conditional entropy = Joint Entropy - Entropy of X
    H(Y|X) = H(Y;X) - H(X)
    Reference: https://en.wikipedia.org/wiki/Conditional_entropy
    """
    return joint_entropy(Y, X) - entropy(X)

#Uncertainty Coefficient
def theil_u(X, Y):
    """
    Reference: https://en.wikipedia.org/wiki/Uncertainty_coefficient
    """
    h_X = entropy(X)
    h_XY = conditional_entropy(X, Y)
    return (h_X - h_XY) / h_X 

In [29]:
vaX_preds = predict_last(vaX_ids, model)


 31%|███       | 271/872 [00:26<00:58, 10.35it/s]


 64%|██████▎   | 554/872 [00:54<00:30, 10.49it/s]


 91%|█████████▏| 797/872 [01:20<00:10,  7.25it/s]


100%|██████████| 872/872 [01:28<00:00,  9.80it/s]


In [58]:
theil_u(vaY, vaX_preds)

0.23951832550975827

In [31]:
vaX_preds_last_2 = predict_topk_last(vaX_ids, model, k=2)


 27%|██▋       | 238/872 [00:25<01:04,  9.89it/s]


 54%|█████▍    | 469/872 [00:51<00:49,  8.22it/s]


 82%|████████▏ | 714/872 [01:18<00:13, 11.44it/s]


100%|██████████| 872/872 [01:34<00:00,  9.26it/s]


In [59]:
theil_u(vaY, vaX_preds_last_2)

0.40554815783377207

In [60]:
vaX_preds_last_3 = predict_topk_last(vaX_ids, model, k=3)
theil_u(vaY, vaX_preds_last_3)


 35%|███▌      | 308/872 [00:23<00:44, 12.70it/s]


 71%|███████   | 616/872 [00:48<00:19, 13.14it/s]


100%|██████████| 872/872 [01:07<00:00, 12.91it/s]


0.6228252421733184